In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras import models
import tensorflow_addons as tfa

In [2]:
class_index = {0: 'angry', 1: 'fear', 2: 'happy',3: 'neutral', 4: 'sad'}
filenames_with_path = tf.io.gfile.glob('data/TIL_test_normalized/*')
filenames = os.listdir('data/TIL_test_normalized/')
sample_rate = 16000


@tf.function(jit_compile=False)
def preprocess_val(filepath):
      wav = tf.io.read_file(filepath)
      wav, sr = tf.audio.decode_wav(wav, desired_channels=1)
      wav = tf.squeeze(wav, axis=-1)
      if no_norm: wav *= 32768.0
      sr = tf.cast(sr, dtype=tf.int64)
      wav = tfio.audio.resample(wav, rate_in=sr, rate_out=sample_rate)
      spectrogram = tfio.audio.spectrogram(wav, nfft=512, window=512, stride=256)
      mel_spectrogram = tfio.audio.melscale(spectrogram, rate=sample_rate, mels=128, fmin=80, fmax=7600)

      # Augmentations
      # mel_spectrogram = tfio.audio.freq_mask(mel_spectrogram, param=10)
      # mel_spectrogram = tfio.audio.time_mask(mel_spectrogram, param=10)

      # Resizing
      mel_spectrogram = tf.expand_dims(mel_spectrogram, axis=-1)
      mel_spectrogram = tf.image.resize_with_pad(mel_spectrogram, target_height=224, target_width=224, method=tf.image.ResizeMethod.BILINEAR)  # any sharper resizing method produce black gaps
      mel_spectrogram = tf.squeeze(tf.stack([mel_spectrogram, mel_spectrogram, mel_spectrogram], axis=-1))
      mel_spectrogram.set_shape([224, 224, 3])
      return mel_spectrogram


no_norm = True
test_ds = tf.data.Dataset.from_tensor_slices(filenames_with_path).map(preprocess_val, num_parallel_calls=-1)
test_ds = test_ds.batch(32).cache().prefetch(-1)

In [3]:
opt = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=5e-5)
opt = tfa.optimizers.Lookahead(opt)

model1 = models.load_model('models/regnetx032_18epoch_TILonly_f1_8848_ranger/', compile=False, custom_objects={'Addons>Lookahead': opt})
#model2 = models.load_model('models/regnetx016_19epoch_f1_7735_ranger//', compile=False, custom_objects={'Addons>Lookahead': opt})  # this one seems better

In [4]:
preds1 = model1.predict(test_ds)
preds1 = tf.nn.softmax(preds1, axis=-1)
# preds2 = model2.predict(test_ds)
# preds2 = tf.nn.softmax(preds2, axis=-1)

19/19 [==============================] - 7s 122ms/step


In [5]:
pred = 1 * preds1
pred_classes = tf.argmax(pred, axis=-1).numpy()
pred_classes = [class_index[i] for i in pred_classes]
pred_df = pd.DataFrame({'filename': filenames, 'pred': pred_classes})
pred_df.to_csv('submission.csv', index=False, header=None)

In [97]:
print(f"Accuracy: {(len(pred_df[pred_df['pred'] == 'sad']) / len(pred_df)) * 100}%")

Accuracy: 53.714285714285715%
